# Manual CPA Attack

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA
* CWNANO

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
#CRYPTO_TARGET = 'TINYAES128C'
CRYPTO_TARGET = 'NONE'
num_traces = 1500
CHECK_CORR = False

In [2]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/willGIFTCPA
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-base-CWLITEARM.hex
rm -f -- simpleserial-base-CWLITEARM.eep
rm -f -- simpleserial-base-CWLITEARM.cof
rm -f -- simpleserial-base-CWLITEARM.elf
rm -f -- simpleserial-base-CWLITEARM.map
rm -f -- simpleserial-base-CWLITEARM.sym
rm -f -- simpleserial-base-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-base.s crypto.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- simpleserial-base.d crypto.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- simpleserial-base.i crypto.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simpl

## The CPA Attack Theory

As a background on the CPA attack, please see the section [Correlation Power Analysis](https://wiki.newae.com/Correlation_Power_Analysis). It's also suggested that you complete Tutorial B1, since that will introduce Jupyter and show how to interface with ChipWhipserer using Python. It's assumed you've read that section and completed the tutorial and come back to this. Ok, you've done that? Good let's continue.

Assuming you **actually** read that, it should be apparent that there is a few things we need to accomplish:

1. Getting some power traces of our target while it's performing AES encryption.
1. Reading the data, which consists of the analog waveform (trace) and input text sent to the encryption core
1. Making the power leakage model, where it takes a known input text along with a guess of the key byte
1. Implementing the correlation equation, and then looping through all the traces
1. Ranking the output of the correlation equation to determine the most likely key.

This tutorial will deal with both recording power traces using ChipWhisperer and breaking them using a CPA attack.

## Capturing Power Traces

Capturing power traces will be very similar to previous tutorials, except this time we'll be using a loop to capture multiple traces, as well as numpy to store them. It's not necessary, but we'll also plot the trace we get using `bokeh`.

### Setup

We'll use some helper scripts to make setup and programming easier. If you're using an XMEGA or STM (CWLITEARM) target, binaries with the correct should be setup for you:

In [3]:
%run "Helper_Scripts/Setup_Generic.ipynb"

Serial baud rate = 38400


In [4]:
fw_path = '../hardware/victims/firmware/willGIFTCPA/simpleserial-base-{}.hex'.format(PLATFORM)

In [5]:
cw.program_target(scope, prog, fw_path)

Serial baud rate = 115200
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5455 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5455 bytes
Serial baud rate = 38400


### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and appends our new trace to the `traces[]` list. At the end, we convert the trace data to numpy arrays, since that's what we'll be using for analysis.

In [13]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()

num_traces = 2000
scope.adc.samples = 23000
scope.adc.offset = 5250

traces = []

for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    traces.append(trace)

#Convert traces to numpy arrays
trace_array = np.asarray([trace.wave for trace in traces])  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])  # for fixed key, these keys are all the same
textout_array = np.asarray([trace.textout for trace in traces])

#print (textout_array[1])
#for b in textout_array[18]: print("%02x " % b, end="")

c:\users\crypto\chipwhisperer5_64\chipwhisperer\software\chipwhisperer\__init__.py:289: UserWarning: Timeout happened during capture
  warnings.warn("Timeout happened during capture")


Now that we have our traces, we can also plot them using Bokeh:

In [7]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

xrange = range(len(traces[0].wave))
p.line(xrange, traces[2].wave, line_color="red")
show(p)

Loading BokehJS ...

In [13]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Trace Analysis

### Using the Trace Data

Now that we have some traces, let's look at what we've actually recorded. Looking at the earlier parts of the script, we can see that the trace data is in `trace_array`, while `textin_array` stores what we sent to our target to be encrypted. For now, let's get some basic information (the total number of traces, as well as the number of sample points in each trace) about the traces, since we'll need that later:

In [14]:
numtraces = np.shape(trace_array)[0] #total number of traces
numpoints = np.shape(trace_array)[1] #samples per trace
print(numtraces)
print(numpoints)

1751
23000


For the analysis, we'll need to loop over every byte in the key we want to attack, as well as every trace:
```python
for bnum in range(0, 16):
    for tnum in range(0, numtraces):
        pass
```
Though we didn't loop over them, note that each trace is made up of a bunch of sample points.
Let's take a closer look at AES so that we can replace that `pass` with some actual code.

### Calculating Correlation

Now that we have some power traces of our target that we can use, we can move on to the next steps of our attack. Looking way back to how AES works, remember we are effectively attemping to target the position at the bottom of this figure:

![title](https://wiki.newae.com/images/7/71/Sbox_cpa_detail.png)

The objective is thus to determine the output of the S-Box, where the S-Box is defined as follows:

In [9]:
sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

GIFTSbox = ( 0x1, 0xa, 0x4, 0xc, 0x6, 0xf, 0x3, 0x9, 0x2, 0xd, 0xb, 0x7, 0x5, 0x0, 0x8, 0xe)

GIFTPbox64 =             ( 0,  17, 34, 51, 48, 1,  18, 35, 32, 49, 2,  19, 16,
                           33, 50, 3,  4,  21, 38, 55, 52, 5,  22, 39, 36, 53,
                           6,  23, 20, 37, 54, 7,  8,  25, 42, 59, 56, 9,  26,
                           43, 40, 57, 10, 27, 24, 41, 58, 11, 12, 29, 46, 63,
                           60, 13, 30, 47, 44, 61, 14, 31, 28, 45, 62, 15 )

#print( GIFTSbox[0])

def BitToByte(input):
    return (int)((input)/8)

def NibbleToByte(input):
    return (int)((input)/2)

def ByteInputSBoxGIFT(input):
    temp = GIFTSbox[(input & 0xf)]
    temp2 = GIFTSbox[((input >> 4) & 0xf)]
    return (temp2 << 4) | temp

def ApplySBox64(input):
    for i in range(8):
        input[i] = ByteInputSBoxGIFT(input[i])
    return input

def ApplyPLayer(input):
    #print ("player function input " , input)
    temp = []
    for i in range(8):
        temp.append(input[i])
        temp[i] = 0 #Used to create an empty byte array of length 8
        #print("byte " , i , " has value " , input[i] )
    for i in range(64):
        tempBit = ((input[BitToByte(i)]) >> ((i%8))) & 0x1
        #print (tempBit)
        pVal = GIFTPbox64[i]
        temp[BitToByte(pVal)] |= (tempBit << (pVal%8))
    return temp

def Invert(input):
    temp = []
    for i in range(8):
        temp.append(input[7-i])
    return temp

def GIFT64PTtoIntermediate1(pt):  #value inbetween 1st pLayer and 1st Add round key
    #temp = Invert(pt)
    temp = []
    for i in range(8):
        temp.append(pt[i])
    temp = ApplySBox64(temp)
    temp = ApplyPLayer(temp)
    return temp

def GIFT64PTtoIntermediate2(pt, RK1):
    temp = GIFT64PTtoIntermediate1(pt)
    for i in range(8):
        temp[i] = temp[i] ^ RK1[i]
    temp = GIFT64PTtoIntermediate1(temp)
    return temp

def GIFT64PTtoIntermediate3(pt, RK1, RK2):
    temp = GIFT64PTtoIntermediate2(pt, RK1)
    for i in range(8):
        temp[i] = temp[i] ^ RK2[i]
    temp = GIFT64PTtoIntermediate1(temp)
    return temp

def GIFT64PTtoIntermediate4(pt, RK1, RK2, RK3):
    temp = GIFT64PTtoIntermediate3(pt, RK1, RK2)
    for i in range(8):
        temp[i] = temp[i] ^ RK3[i]
    temp = GIFT64PTtoIntermediate1(temp)
    return temp

def applyGuess(index, buffer, subKeyGuess):
    #print(NibbleToByte(index))
    temp = (buffer[NibbleToByte(index)] >> ((index % 2)*4)) & 0x0f
    temp = temp ^ subKeyGuess
    temp = GIFTSbox[temp]
    #print(temp)
    #temp = buffer[0] & 0xf
    #temp = temp ^ subKeyGuess
    #temp = GIFTSbox[temp]
    return temp

def GuessToRoundKeyFormat(arr):
    retArr = [0]*8
    #print ("length return val " , len(retArr))
    #print ("length arr " , len(arr))
    for i in range(8): 
        #print (i)
        retArr[i] = arr[2*i] | (arr[(2*i+1)] << 4)
    return retArr


    
def ExtractV(RK):
    retArr = [0]*2
    for i in range(8):
        bitLow = (RK[i] & 0b1)
        bitHigh = ((RK[i] >> 4) & 0b1)
        concatVal = bitLow | (bitHigh << 1)
        retArr[(int)(i/4)] |= (concatVal << ((i%4)*2))
    return retArr

def ExtractU(RK):
    retArr = [0]*2
    for i in range(8):
        bitLow = ( (RK[i]>> 1) & 0b1)
        bitHigh = ((RK[i] >> 5) & 0b1)
        concatVal = bitLow | (bitHigh << 1)
        retArr[(int)(i/4)] |= (concatVal << ((i%4)*2))
    return retArr

def ReverseKeyScheduler(RK1, RK2, RK3,RK4):
    retArr = [0] * 16
    
    V0 = ExtractV(RK1)
    V1 = ExtractV(RK2)
    V2 = ExtractV(RK3)
    V3 = ExtractV(RK4)
    U0 = ExtractU(RK1)
    U1 = ExtractU(RK2)
    U2 = ExtractU(RK3)
    U3 = ExtractU(RK4)
    
    retArr[0] = V0[0]
    retArr[1] = V0[1]
    retArr[2] = U0[0]
    retArr[3] = U0[1]
    retArr[4] = V1[0]
    retArr[5] = V1[1]
    retArr[6] = U1[0]
    retArr[7] = U1[1]
    retArr[8] = V2[0]
    retArr[9] = V2[1]
    retArr[10] = U2[0]
    retArr[11] = U2[1]
    retArr[12] = V3[0]
    retArr[13] = V3[1]
    retArr[14] = U3[0]
    retArr[15] = U3[1]
    
    return retArr
        
#print (ExtractV(RK1))
#print (ExtractU(RK1))

Thus we need to write a function taking a single byte of input, a single byte of the guessed key, and return the output of the S-Box:

In [11]:
def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]

Finally, remember we want the Hamming Weight of the guess. Our assumption is that the system is leaking the Hamming Weight of the output of that S-Box. As a dumb solution, we could first convert every number to binary and count the 1's:

```python
>>> bin(0x1F)
'0b11111'
>>> bin(0x1F).count('1')
5
```
This will ultimately be fairly slow. Instead we make a lookup table using this idea:

In [10]:
HW = [bin(n).count("1") for n in range(0, 16)]

### Performing the Check

Remember the objective is to calculate the following:
$$r_{i,j} = \frac{\sum_{d=1}^{D}[(h_{d,i} - \bar{h_i})(t_{d,j}-\bar{t_j})]}{\sqrt{\sum_{d=1}^D(h_{d,i}-\bar{h_i})^2\sum_{d=1}^D(t_{d,j}-\bar{t_j})^2}}$$

Where

| **Equation** | **Python Variable** | **Value**  | 
|--------------|---------------------|------------|
|  d           |       tnum          | trace number |
|  i           |       kguess        | subkey guess |
| j | j index trace point | sample point in trace |
| h | hypint | guess for power consumption | 
| t | traces | traces | 

It can be noticed there is effectively three sums, all sums are done over all traces. For this initial implementation we'll be explicitly calculating some of these sums, although it's faster to use NumPy to calculate across large arrays. We'll convert those three summations into variables, turning the equation into this format:

$$r_{i,j}=\frac{sumnum}{\sqrt{snumden1 * sumden2}}$$

Where:

$$sumnum = \sum_{d=1}^{D}[(h_{d,i} - \bar{h_i})(t_{d,j}-\bar{t_j})]$$

$$sumden1 = \sum_{d=1}^D(h_{d,i}-\bar{h_i})^2$$

$$sumden2 = \sum_{d=1}^D(t_{d,j}-\bar{t_j})^2$$

Looking at this, we can see that we'll need $\bar{h_i}$ and $\bar{t_j}$, so let's start by building some code that will give us those. Looking at the CPA tutorial, we can see that $h_{d,i}$ is just our guess for the power consumption in trace $d$ for subkey $i$. We can get that easily using the `HW` array and `intermediate()` function we defined earlier:

```python
for bnum in range(0, 16):
    cpaoutput = [0]*256
    maxcpa = [0]*256
    for kguess in range(0, 256):
        hyp = np.zeros(numtraces)
        for tnum in range(0, numtraces):
            hyp[tnum] = HW[intermediate(pt[tnum][bnum], kguess)]
```

Now we can get $\bar{h_i}$:
```python
meanh = np.mean(hyp, dtype=np.float64)
```

and $\bar{t_j}$ is just the mean of all of our traces:
```python
meant = np.mean(traces, axis=0, dtype=np.float64)
```

Next, let's move on to calculating the whole sums using $h_{d,i}$ and $t_{d,j}$ and the values we just calculated:

```python
#For each trace, do the following
for tnum in range(numtraces):
    hdiff = (hyp[tnum] - meanh)
    tdiff = traces[tnum,:] - meant

    sumnum = sumnum + (hdiff*tdiff)
    sumden1 = sumden1 + hdiff*hdiff 
    sumden2 = sumden2 + tdiff*tdiff
```

We can now get the correlation for each of our subkey guesses, which we'll call `cpaoutput[]`:

```python
cpaoutput[kguess] = sumnum / np.sqrt( sumden1 * sumden2 )
```

We're almost done! All that's left is to use that correlation to figure out which subkey best matches our power traces. First off, we only care about absolute value of the correlation (that there is a linear correlation), not sign. Additionally, though this didn't factor into our correlation calculation, remember that each trace was actually made up of a bunch of sample points. This means that what we actually have is the correlation of each subkey guess to each sample point. Typically only a few points in the trace are correlating, and it's the maximum across the entire trace we are concerned with, so we can pick our correlation for each subkey by:

```python
maxcpa[kguess] = max(abs(cpaoutput[kguess]))
```

Finally, we can find the subkey that best matches our data by finding the one with the biggest correlation:

```python
bestguess[bnum] = np.argmax(maxcpa)
```

Putting it all together:

#### Finished Script

In [16]:
import numpy as np
from tqdm import tqdm

sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]

HW = [bin(n).count("1") for n in range(0, 16)]

numtraces = np.shape(trace_array)[0] #total number of traces
numpoint = np.shape(trace_array)[1] #samples per trace
#numpoint = 22000

pt = textin_array
knownkey = traces[0].key
cparefs = [0] * 16
bestguess = [0]*16

for bnum in tqdm(range(0, 16), desc='Attacking subkeys from Round Key 1'):
    cpaoutput = [0] * 16
    maxcpa = [0] * 16
    for kguess in range(0, 16):

        # Initialize arrays &amp; variables to zero
        sumnum = np.zeros(numpoint)
        sumden1 = np.zeros(numpoint)
        sumden2 = np.zeros(numpoint)

        hyp = np.zeros(numtraces)
        for tnum in range(0, numtraces):
            #hyp[tnum] = HW[intermediate(pt[tnum][bnum], kguess)]
            tempNum = GIFT64PTtoIntermediate1(textin_array[tnum])
            appliedGuess = applyGuess(bnum, tempNum, kguess)
            hyp[tnum] = HW[appliedGuess]

        # Mean of hypothesis
        meanh = np.mean(hyp, dtype=np.float64)
        #print(meanh)

        # Mean of all points in trace
        meant = np.mean(trace_array, axis=0, dtype=np.float64)
        #print("here")
        #print(len(meant))

        # For each trace, do the following
        for tnum in range(0, numtraces):
            hdiff = (hyp[tnum] - meanh)
            tdiff = trace_array[tnum, :] - meant
            #print("tdiff")
            #print("len tdiff " , len(tdiff))
            #print(tdiff)

            sumnum = sumnum + (hdiff * tdiff)
            sumden1 = sumden1 + hdiff * hdiff
            sumden2 = sumden2 + tdiff * tdiff
            #print("len sumnum ", len(sumnum))

        cpaoutput[kguess] = sumnum / np.sqrt(sumden1 * sumden2)
        maxcpa[kguess] = max(abs(cpaoutput[kguess]))
       # print("Subkey guess " , kguess, " had correlation value " , maxcpa[kguess])

    bestguess[bnum] = np.argmax(maxcpa)
    cparefs[bnum] = np.argsort(maxcpa)[::-1]

RK1 = GuessToRoundKeyFormat(bestguess)
#rint(RK1)
print("Prediction for Round Key 1")
for b in RK1: print("%02x " % b, end="")
#print("Best Key Guess: ", end="")
#for b in bestguess: print("%01x " % b, end="")

cparefs = [0] * 16
bestguess = [0]*16

for bnum in tqdm(range(0, 16), desc='Attacking subkeys from Round Key 2'):
    cpaoutput = [0] * 16
    maxcpa = [0] * 16
    for kguess in range(0, 16):

        # Initialize arrays &amp; variables to zero
        sumnum = np.zeros(numpoint)
        sumden1 = np.zeros(numpoint)
        sumden2 = np.zeros(numpoint)

        hyp = np.zeros(numtraces)
        for tnum in range(0, numtraces):
            #hyp[tnum] = HW[intermediate(pt[tnum][bnum], kguess)]
            tempNum = GIFT64PTtoIntermediate2(textin_array[tnum], RK1)
            appliedGuess = applyGuess(bnum, tempNum, kguess)
            hyp[tnum] = HW[appliedGuess]

        # Mean of hypothesis
        meanh = np.mean(hyp, dtype=np.float64)

        # Mean of all points in trace
        meant = np.mean(trace_array, axis=0, dtype=np.float64)

        # For each trace, do the following
        for tnum in range(0, numtraces):
            hdiff = (hyp[tnum] - meanh)
            tdiff = trace_array[tnum, :] - meant
            

            sumnum = sumnum + (hdiff * tdiff)
            sumden1 = sumden1 + hdiff * hdiff
            sumden2 = sumden2 + tdiff * tdiff

        cpaoutput[kguess] = sumnum / np.sqrt(sumden1 * sumden2)
        maxcpa[kguess] = max(abs(cpaoutput[kguess]))

    bestguess[bnum] = np.argmax(maxcpa)
    cparefs[bnum] = np.argsort(maxcpa)[::-1]
    
RK2 = GuessToRoundKeyFormat(bestguess)
print("Prediction for Round Key 2")
for b in RK2: print("%02x " % b, end="")
#print("Best Key Guess: ", end="")
#for b in bestguess: print("%01x " % b, end="")

cparefs = [0] * 16
bestguess = [0]*16

for bnum in tqdm(range(0, 16), desc='Attacking subkeys from Round Key 3'):
    cpaoutput = [0] * 16
    maxcpa = [0] * 16
    for kguess in range(0, 16):

        # Initialize arrays &amp; variables to zero
        sumnum = np.zeros(numpoint)
        sumden1 = np.zeros(numpoint)
        sumden2 = np.zeros(numpoint)

        hyp = np.zeros(numtraces)
        for tnum in range(0, numtraces):
            #hyp[tnum] = HW[intermediate(pt[tnum][bnum], kguess)]
            tempNum = GIFT64PTtoIntermediate3(textin_array[tnum], RK1, RK2)
            appliedGuess = applyGuess(bnum, tempNum, kguess)
            hyp[tnum] = HW[appliedGuess]

        # Mean of hypothesis
        meanh = np.mean(hyp, dtype=np.float64)

        # Mean of all points in trace
        meant = np.mean(trace_array, axis=0, dtype=np.float64)

        # For each trace, do the following
        for tnum in range(0, numtraces):
            hdiff = (hyp[tnum] - meanh)
            tdiff = trace_array[tnum, :] - meant

            sumnum = sumnum + (hdiff * tdiff)
            sumden1 = sumden1 + hdiff * hdiff
            sumden2 = sumden2 + tdiff * tdiff

        cpaoutput[kguess] = sumnum / np.sqrt(sumden1 * sumden2)
        maxcpa[kguess] = max(abs(cpaoutput[kguess]))

    bestguess[bnum] = np.argmax(maxcpa)
    cparefs[bnum] = np.argsort(maxcpa)[::-1]
    
RK3 = GuessToRoundKeyFormat(bestguess)
print("Prediction for Round Key 3")
for b in RK3: print("%02x " % b, end="")
#print("Best Key Guess: ", end="")
#for b in bestguess: print("%01x " % b, end="")

cparefs = [0] * 16
bestguess = [0]*16

for bnum in tqdm(range(0, 16), desc='Attacking subkeys from Round Key 4'):
    cpaoutput = [0] * 16
    maxcpa = [0] * 16
    for kguess in range(0, 16):

        # Initialize arrays &amp; variables to zero
        sumnum = np.zeros(numpoint)
        sumden1 = np.zeros(numpoint)
        sumden2 = np.zeros(numpoint)

        hyp = np.zeros(numtraces)
        for tnum in range(0, numtraces):
            #hyp[tnum] = HW[intermediate(pt[tnum][bnum], kguess)]
            tempNum = GIFT64PTtoIntermediate4(textin_array[tnum], RK1, RK2, RK3)
            appliedGuess = applyGuess(bnum, tempNum, kguess)
            hyp[tnum] = HW[appliedGuess]

        # Mean of hypothesis
        meanh = np.mean(hyp, dtype=np.float64)

        # Mean of all points in trace
        meant = np.mean(trace_array, axis=0, dtype=np.float64)

        # For each trace, do the following
        for tnum in range(0, numtraces):
            hdiff = (hyp[tnum] - meanh)
            tdiff = trace_array[tnum, :] - meant

            sumnum = sumnum + (hdiff * tdiff)
            sumden1 = sumden1 + hdiff * hdiff
            sumden2 = sumden2 + tdiff * tdiff

        cpaoutput[kguess] = sumnum / np.sqrt(sumden1 * sumden2)
        maxcpa[kguess] = max(abs(cpaoutput[kguess]))

    bestguess[bnum] = np.argmax(maxcpa)
    cparefs[bnum] = np.argsort(maxcpa)[::-1]
    
RK4 = GuessToRoundKeyFormat(bestguess)
print("Prediction for Round Key 4")
for b in RK4: print("%02x " % b, end="")
#print("Best Key Guess: ", end="")
#for b in bestguess: print("%01x " % b, end="")

guessKey = ReverseKeyScheduler(RK1, RK2, RK3,RK4)
print ("\nPredicted Key")
for b in guessKey: print("%02x " % b, end="")

Attacking subkeys from Round Key 1: 100%|██████████| 16/16 [01:42<00:00,  6.41s/it]


Prediction for Round Key 1
1b 12 12 00 30 13 13 81 

Attacking subkeys from Round Key 2: 100%|██████████| 16/16 [02:20<00:00,  8.85s/it]


Prediction for Round Key 2
a8 10 12 22 30 13 30 b0 

Attacking subkeys from Round Key 3: 100%|██████████| 16/16 [03:01<00:00, 11.07s/it]


Prediction for Round Key 3
9b 1a 12 10 11 21 11 b1 

Attacking subkeys from Round Key 4: 100%|██████████| 16/16 [03:40<00:00, 13.46s/it]


Prediction for Round Key 4
ab ba 00 02 11 33 22 91 
Predicted Key
2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c 

### Checking Our Guess

Since we were the ones who decided the key (and kept track of it in knownkey), we can check if our guess was right:

In [17]:
for b in knownkey: print("%02x "%b, end="")
guessKey = ReverseKeyScheduler(RK1, RK2, RK3,RK4)
print("\n")
if all([known_byte == guess_byte for known_byte, guess_byte in zip(knownkey, guessKey)]):
    print("Guess was right")
else:
    print("Guess was wrong")

2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c 

Guess was right


We can do even better by calculating the Partial Guessing Entropy.

The Partial Guessing Entropy (PGE) is a useful metric of where the correct answer is ranked. This requires us to know the actual encryption key used during operation, which we do know! 

Certain attacks will use different keys during the acqusition period, meaning the a list of all the known keys is required since there isn't a constant key. In our case, we already have access in `knownkey`.

Previously, we simply printed the maximum output for each subkey as follows:

```python
#Find maximum value of key
bestguess[bnum] = np.argmax(maxcpa)
```

To sort the list of CPA output's, we'll use the `argsort()` function from NumPy. This will output a list where the first element is the index of the lowest value, next element is the index of the next-highest element, etc. Because in our input list the `maxcpa` vector's indexes correspond to the key guess, this allows us to know where the keys are. We reverse that sorted list to put the first element as the maximum CPA output:

```python
cparefs = np.argsort(maxcpa)[::-1]
```

Finally, the Partial Guessing Entropy is simply the location of the known correct key byte inside that array. We can find that with the `.index()` function:

```python
print cparefs.index(0x2B)
```

Where the correct key should of course come from our `knownkey` variable instead of being hard-coded. Pulling it all together:

In [32]:
pge = [0]*16
for bnum in range(0, 16):
    #Find maximum value of key
    #Find PGE
    pge[bnum] = list(cparefs[bnum]).index(knownkey[bnum])
    
print("PGE: ", end="")
for i in pge:
    print(i, end="")
print("")

PGE: 0000000000000000


### Improvements

The implementation of the correlation function runs as a loop over all traces. Ideally we'd like to implement this as a 'online' calculation; that is, we can add a trace in, observe the output, add another trace in, observe the output, etc. When generating plots of the Partial Guessing Entropy (PGE) vs. number of traces this is greatly preferred, since otherwise we need to run the loop many times!

We can use an alternate form of the correlation equation, which explicitly stores sums of the variables. This is easier to perform online calculation with, since when adding a new trace it's simple to update these sums. This form of the equation looks like:

$$r_{i,j} = \frac{D\sum_{d=1}^{D}h_{d,i}t_{d,j}-\sum_{d=1}^{D}h_{d,i}\sum_{d=1}^{D}t_{d,j}}{\sqrt{((\sum_{d=1}^Dh_{d,i})^2-D\sum_{d=1}^Dh_{d,i}^2)-((\sum_{d=1}^Dt_{d,j})^2-D\sum_{d=1}^Dh_{d,j}^2)}}$$

## Tests

In [17]:
assert all([known_byte == guess_byte for known_byte, guess_byte in zip(knownkey, bestguess)]), "Failed to break encryption key\nGot: {}\nExpected: {}\n".format(knownkey, bestguess)

In [34]:
assert not (np.all(pge)), "PGE not zero for byte: {}".format(pge)